In [1]:
from langchain.llms import OpenAI, OpenAIChat

import os


os.environ['OPENAI_API_KEY'] = ''

In [18]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS 

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [23]:
# location of the pdf file/files. 
doc_reader = PdfReader('buildings-12-01787-v3.pdf')

# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [24]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [25]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

docsearch = FAISS.from_texts(texts, embeddings)

In [26]:
chain = load_qa_chain(OpenAI(), 
                      chain_type="stuff") # we are going to stuff all the docs in at once

# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [27]:
query = "explain me in detail what the article states. it should everythong. even mathematical explaination"

docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" This article is discussing a system for controlling a chilled water system using reinforcement learning (RL). The authors explain the workflow of the system, which includes using a coefficient of performance (COP) as a measure of the energy-saving performance of the HVAC system. The COP is calculated by Equation (5). The authors also discuss how Deep Forest (DF) is used to obtain feature vectors. Finally, the article explains how Markov's Decision Process is used to model RL problems. The MDP is defined by a tuple of states, actions, transition functions, reward functions, and a discount factor. The goal is to maximize a cumulative numerical reward, as shown in Equation (1)."